# TorchRL 学习笔记

> 汇总 TorchRL 的核心概念、模块结构、典型训练流程及关键 API，帮助快速在 PyTorch 生态中构建强化学习（RL）实验。TorchRL 是 Meta AI 开源的 RL 库，提供环境封装、数据管线、策略训练、推理与部署组件。

## TorchRL 概览
- **定位**：PyTorch 原生的 RL 库，覆盖环境抽象、数据收集、策略学习、评估与部署。
- **设计目标**：
  - 模块化：环境、策略、数据管线等组件可组合使用。
  - 可扩展：支持多 GPU、多环境并行收集、分布式训练。
  - 实用性：内置常见算法与工具，兼容 Gymnasium/DeepMind 控件等生态。
- **核心组件**：
  - 环境接口 (`EnvBase`, `TransformedEnv`)
  - 数据迭代 (`TensorDict`, `TensorDictReplayBuffer`)
  - 策略/模块 (`rl.modules`)
  - 训练循环 (`rl.trainers`, `collectors`)
  - 评估与部署工具 (`RLTrainer`, `policy.eval`, TorchScript/ONNX 导出)

## 安装与检查
- **依赖**：PyTorch >= 2.1（建议最新版）、torchrl、gymnasium（或其它环境库）。
- **安装示例**：
  ```bash
  pip install torchrl
  pip install gymnasium
  pip install gymnasium[classic-control]
  ```
- TorchRL npm 包发布频率较快，建议关注 <https://pytorch.org/rl/> 的安装说明。

In [ ]:
# 检查 TorchRL 安装状态
try:
    import torch, torchrl
    from torchrl.envs import GymnasiumEnv
    print('PyTorch 版本:', torch.__version__)
    print('TorchRL 版本:', torchrl.__version__)
    env = GymnasiumEnv('CartPole-v1', device='cpu')
    td = env.reset()
    print('环境初始 observation keys:', td.keys())
except Exception as err:
    print('TorchRL 或相关依赖未正常安装 ->', err)
    print('请按文档安装 torchrl / gymnasium 等依赖')

## 核心概念
### TensorDict
- TorchRL 以 `TensorDict` 作为数据容器，统一存放观测、动作、奖励等字段。
- 支持批处理、并行维度、设备迁移，与 PyTorch Tensor 操作高度兼容。
- 常用方法：`td.get(...)`、`td.set(...)`、`td.to(device)`、`td.batch_size`。

### 环境 (EnvBase, TransformedEnv)
- `EnvBase`：统一步进接口，返回 TensorDict。
- `GymnasiumEnv`, `DMEnvWrapper`：对第三方环境的封装。
- `TransformedEnv`：叠加一系列 `Transform`（归一化、裁剪、奖励重塑等），便于预处理。

### 策略与模块
- `rl.modules`：提供 `Actor`, `ValueOperator`, `DistributionalActor` 等模块化结构。
- 支持与 `nn.Module` 集成，实现自定义策略网络。

### 数据收集与缓冲
- `Collectors`：如 `SyncDataCollector`, `MultiaSyncDataCollector` 并行样本采集。
- `ReplayBuffer`：`TensorDictReplayBuffer`、`LazyTensorStorage` 等，用于经验回放。

### 训练器 (Trainers)
- `RLTrainer`：组织收集、优化、评估流程。
- 内置多种常用算法：DQN、A2C、PPO、SAC、TD3 等，位于 `torchrl.trainers.helpers`。

## 快速入门示例：DQN 训练 CartPole

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torchrl.envs import GymnasiumEnv, TransformedEnv, DoubleToFloat
from torchrl.data.replay_buffers import TensorDictReplayBuffer, LazyTensorStorage
from torchrl.collectors import SyncDataCollector
from torchrl.modules import QValueModule, EGreedyModule
from torchrl.objectives import DQNLoss
from torchrl.objectives.value import DQNLossDefaultActorCritic

# 环境定义
env = TransformedEnv(
    GymnasiumEnv('CartPole-v1', device='cpu'),
    DoubleToFloat(),
)
td = env.reset()
obs_size = td.observation.shape[-1]
num_actions = env.action_spec.space.n

# Q 网络
def make_qnet():
    return nn.Sequential(
        nn.Linear(obs_size, 128), nn.ReLU(),
        nn.Linear(128, 128), nn.ReLU(),
        nn.Linear(128, num_actions)
    )

qnet = QValueModule(make_qnet())
policy = EGreedyModule(qnet, eps_init=0.1)

# 经验回放
buffer = TensorDictReplayBuffer(
    storage=LazyTensorStorage(10000),
    batch_size=128,
)

# 收集器
collector = SyncDataCollector(
    env,
    policy,
    frames_per_batch=200,
    total_frames=10_000,
    device='cpu'
)

optimizer = Adam(qnet.parameters(), lr=3e-4)
loss_module = DQNLoss(
    value_network=qnet,
    value_type='qvalue',
    double_dqn=True,
    gamma=0.99
)

for i, data in enumerate(collector):
    buffer.extend(data)
    for _ in range(10):
        sample = buffer.sample()
        loss = loss_module(sample)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if i % 5 == 0:
        print(f'Batch {i}, loss = {loss.item():.4f}')

print('训练完成，测试策略...')
with torch.no_grad():
    td = env.reset()
    total_reward = 0.0
    done = False
    while not done:
        action = policy(td).get('action')
        td = env.step(action)
        reward = td.get('reward').item()
        total_reward += reward
        done = td.get('done').item()
    print('测试回合总奖励:', total_reward)

## 常见算法与模块
- **值函数类**：`DQNLoss`, `TD0Loss`, `A2CLoss`, `PPOLoss`, `SACLoss`, `REDQLoss` 等。
- **策略分布**：`GaussianActor`, `TanhNormal`, `CategoricalActor`。
- **辅助工具**：
  - `torchrl.record.loggers`：TensorBoard、CSV、W&B 等日志。
  - `torchrl.trainers.helpers`：封装了训练构建流程（环境、策略、损失、日志器等）。
  - `rl.utils`：seed 控制、探索策略、TDError 计算等。

常用模块结构：
```
torchrl
 ├── data             # TensorDict、ReplayBuffer
 ├── envs             # 环境与 Transform
 ├── modules          # 策略/价值模块
 ├── objectives       # 损失函数
 ├── collectors       # 数据采集器
 ├── trainers         # 训练器及 helper
 └── record           # 日志与分析
```

## 高级主题
- **多环境并行**：`ParallelEnv`, `SerialEnv`, `MultiprocessingEnv`
- **分布式训练**：结合 `torch.distributed`，通过 `MultiaSyncDataCollector` 等实现分布式采集/训练。
- **离线 RL**：支持从离线数据加载 `TensorDict`，结合 `ReplayBuffer` 进行训练。
- **安全与可解释**：利用 `Transforms` 对奖励、动作空间进行裁剪或约束。
- **部署**：策略可导出为 TorchScript/ONNX，用于 C++/移动端部署。

## 参考资料
- 官方文档：<https://pytorch.org/rl/>
- GitHub 仓库：<https://github.com/pytorch/rl>
- 教程与示例：文档中 “Tutorials” 与 “Examples” 栏目。
- 论文参考：TorchRL 白皮书、Meta AI 强化学习相关研究。